In [ ]:
from circuits.utils import (
    othello_hf_dataset_to_generator,
    get_model,
)
from circuits.dictionary_learning.buffer import NNsightActivationBuffer
from circuits.dictionary_learning.dictionary import AutoEncoder, GatedAutoEncoder, AutoEncoderNew, IdentityDict
from circuits.dictionary_learning.evaluation import evaluate
from circuits.utils import get_ae

%load_ext autoreload
%autoreload 2

In [ ]:
layer = 5 # or ""
transcoder = True
device='cuda:0'
repo_dir = '/share/u/can/OthelloUnderstanding'

node_type = "sae_feature"
# node_type = "mlp_neuron"

# ae_group_name = 'mlp_out_sweep_all_layers_panneal_0628'
ae_group_name = 'mlp_transcoder_all_layers_panneal_0628'
# ae_group_name = 'othello_mlp_acts_identity_aes_lines' # with_lines
ae_type = 'p_anneal'

In [ ]:
# model and data parameters
model_name = "Baidicoot/Othello-GPT-Transformer-Lens"
dataset_name = "taufeeque/othellogpt"
context_length = 59
activation_dim = 512  # output dimension of the layer

buffer_size = int(3e4 / 4)
llm_batch_size = 128 # 256 for A100 GPU, 64 for 1080ti
sae_batch_size = 4096
num_tokens = 200_000_000    

# Initialize model, data and activation buffer
model = get_model(model_name, device)
data = othello_hf_dataset_to_generator(
    dataset_name, context_length=context_length, split="train", streaming=True
)
if transcoder:
    io = "in_and_out"
    submodule = model.blocks[layer].mlp
else:
    io = "out"
    # submodule = model.blocks[layer].hook_resid_post # resid_post
    submodule = model.blocks[layer].hook_mlp_out # resid_post
    # submodule = model.blocks[layer].mlp.hook_post # resid_pre

activation_buffer = NNsightActivationBuffer(
    data,
    model,
    submodule,
    n_ctxs=buffer_size,
    ctx_len=context_length,
    refresh_batch_size=llm_batch_size,
    out_batch_size=sae_batch_size,
    io=io,
    d_submodule=activation_dim,
    device=device,
)

In [ ]:
trainer_id = 0

ae = get_ae(layer, node_type, repo_dir, ae_group_name, trainer_id, device=device)
eval_results = evaluate(
    ae,
    activation_buffer,
    max_len=context_length,
    batch_size=sae_batch_size,
    io=io,
    tracer_args={}, #{'scan': False, 'validate':False},
    device=device,
)


In [ ]:
eval_results